In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os

In [2]:
import glob

In [3]:
from Matrix import Matrix
from AuxFunctions import zip_df_cols, PairWiseColumnGroups, trim_index_df

In [4]:
def nband(n):
    '''
    Returns a function that returns the corresponding n_band for x
    nband is a wrapper that parametrises n for a banding function
    Useful for pandas.groupby(nband(n))
    
    >>> [nband(2)(x) for x in range(7)]
    [0, 0, 2, 2, 4, 4, 6]
    '''
    return lambda x: int(x/n)*n

In [57]:
def band_agg_TLD(TLD, n):
    '''Takes a TLD dataframe (distances as index, one column per TLD),
    and returns a TLD dataframe aggregated to bands of n.'''
    TLDband = TLD.index.get_level_values(-1)[1]
    if n < TLDband:
        ErrMsg = '''input n ({}) < TLD band aggregation ({})
            This function cannot be used to disaggregate a TLD'''.format(n, TLDband)
        raise ValueError(ErrMsg)
    TLDn = TLD.groupby(nband(n)).sum() #TLD by bands
    TLDn.index = TLDn.index + n #re-index to top end of each band
    TLDn.at[0,:]=0 #TLD with initial zero value
    return TLDn.sort_index()

In [6]:
def normalize_TLD(TLD):
    '''Normalizes TLD so TLD will contain proportion of trips 
    for each distance band rather thab absolute number of trips.'''
    return TLD.apply(lambda x: x / x.sum())

In [7]:
def trim_index_TLD(TLD, index_names_to_keep='from', inplace=False):
    '''Wrapper for trim_index_df, adapted for TLD.'''
    return trim_index_df(TLD, index_names_to_keep, inplace)

In [8]:
def to_numeric_TLD(TLD):
    '''Returns TLD where strings have been converted to numbers.'''
    tmp_index_names = TLD.index.names
    TLD.index = pd.to_numeric(TLD.index)
    TLD.index.names = tmp_index_names #I can't remmeber now why this is necessary
    return TLD.apply(lambda x: pd.to_numeric(x))

In [9]:
def truncate_TLD(TLD, dist):
    '''Truncates a dataframe based on the index values.'''
    return TLD.loc[TLD.index < dist]

In [111]:
def avgdist(TLD,col,level=-1):
    '''Returns the average distance (weighted average, SUMPRODUCT)
    ofspecified column in TLD'''
    return (TLD[col] * TLD.index.get_level_values(-1)).sum()

In [10]:
def TLD_col(mat, dist_band, dist_col=0):
    '''Returns the Trip-Lenght Distribution of mat, 
    based on dist_col, aggregated by dist_band.'''
    #TODO: implement normalized
    
    if isinstance(dist_col, int):
        dist_col = mat.columns[dist_col]
    
    TLD = mat.copy()
    TLD.ix[:,dist_col] = TLD.ix[:,dist_col].apply(nband(dist_band))
    
    TLD = TLD.groupby(by=dist_col).sum()
    TLD.index = TLD.index + dist_band #top end of each band
    TLD.at[0,:]=0 #fill initial zero value
    return TLD.sort_index()

In [11]:
def TLD_SingleDist(mat, dist, dist_band, dist_col=0):
    '''Returns the Trip-Lenght Distribution of mat, 
    based on distance (dist_col) form dist, aggregated by dist_band.
    mat can have any number of culumns, but only dist_col will be used
    for the TLD. dist_col admits integer and column name.'''
    #TODO: implement normalized
    
    if isinstance(dist_col, int):
        dist_col = dist.columns[dist_col]
    
    df = mat.join(dist.ix[:,[dist_col]]).fillna(0)
    TLD = TLD_col(df, dist_band, dist_col)
    return TLD

In [12]:
def TLD_MultiDist(mat, dist, dist_band):
    '''Returns the Trip-Length Distribution of mat.
    TLD for each mat column will be based on the corresponding
    column from dist (in order). mat and dist must have the same
    number of columns, or just the first distance column will be
    used.'''
    #TODO: implement normalized
    
    if len(mat.columns) != len(dist.columns):
        return TLD_single(mat, dist, dist_band)
    
    dfs = zip_df_cols([mat,dist])
    TLDs = [TLD_col(df, dist_band, 1) for df in dfs]
    
    TLD = pd.DataFrame()
    for xTLD in TLDs:
        TLD = pd.concat([TLD, xTLD], axis=1)
        
    return TLD

In [13]:
def read_EMME_TLD(file):
    '''Returns TLD df from an EMME TLD report file, with columns:
    ['from','to','density_abs','density_norm','cumulative_abs','cumulative_norm']
    '''
    
    # EMME_TLD_cols - in order, position matters
    EMME_TLD_cols = ['from','to','density_abs','density_norm','cumulative_abs','cumulative_norm']

    idx_cols = EMME_TLD_cols[:2]
    data_cols = EMME_TLD_cols[2:]
    
    # RegEx to read EMME format:
    NumberPat = r'-?\.?\d*\.?\d+'
    TLDRowPat = r'(?<=\n)\s*({0})\s+({0})\s+({0})\s+({0})\s+({0})\s+({0})'
    EMMErecord_re = re.compile(TLDRowPat.format(NumberPat))
    
    # Read data
    with open(file, 'r') as f:
        f_content = f.read()
        data = EMMErecord_re.findall(f_content)
            
    # Convert data to DataFrame
    df = pd.DataFrame.from_records(data,
                                   columns=EMME_TLD_cols,
                                   index=idx_cols)
    return df

In [43]:
def read_EMME_TLDs(files):
    '''Reads all TLD reports specified in files
    and returns four DataFrames, with the TLDs combined.
    Recomended: use glob to get the list of files from a pattern.
    Returns one DataFrame for each of the TLD EMME columns:
    ['density_abs','density_norm','cumulative_abs','cumulative_norm']
    '''
    TLDs = [read_EMME_TLD(file) for file in files]
    combinedTLDs = list(PairWiseColumnGroups(TLDs))
    
    filenames = [os.path.basename(file) for file in files]
    for TLD in combinedTLDs:
        TLD.columns = filenames
    density_abs, density_norm, cumulative_abs, cumulative_norm = combinedTLDs
    
    return density_abs, density_norm, cumulative_abs, cumulative_norm

In [144]:
def TLD_to_JPG(TLD, OutputName='TLD.png', ylabel='Trips', units=''):
    '''Produces a graph from TLD, all columns together.
    Includes average distance.'''
    
    plt.clf()
    TLD.plot(title='Trip-Length Distribution', legend=True)
    lgd = plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
                      fancybox=True, ncol=len(TLD.columns))
    plt.xlabel('Dist')
    plt.ylabel(ylabel)

    if units:
        col_label = 'Avg Dist ({})'.format(units)
    else:
        col_label = 'Avg Dist'

    table = plt.table(
        cellText=[['{:,.2f}'.format(avgdist(TLD,col))] for col in TLD],
        colWidths = [0.1],
        rowLabels=[' {} '.format(col) for col in TLD],
        colLabels=[col_label],
        loc='upper right')
    #table.set_fontsize(16)
    table.scale(2, 2)

    oName = OutputName
    plt.savefig(oName, bbox_inches='tight')
    plt.close()

In [145]:
def TLD_cols_to_JPGs(TLD, oFileNamePattern='TLD_{}.png', ylabel='Trips', units=''):
    '''Exports the columns of a TLD dataframe as JPGs,
       naming them based on oFileNamePattern.
       Includes average distance'''
    for col in TLD:
        oFname = oFileNamePattern.format(col)
        TLD_to_JPG(TLD[[col]], oFname, ylabel=ylabel, units=units)

In [17]:
ex_matrixf = os.path.join('example_data', 'ex_matrix_1.csv')
ex_matrix = Matrix(pd.DataFrame.from_csv(ex_matrixf, index_col=[0,1]))
ex_matrix

T
zone_O zone_D       
1001   1001     16.0
       1002      8.0
       1003     16.0
       1004      6.0
       1005      2.0
       1006      6.0
       1007      3.0
       1008      6.0
       1009      6.0
       1010     10.0
       1011      8.0
       1012     21.0
       1013      7.0
       1014     12.0
       1015     27.0
       1016      4.0
       1017     28.0
       1018     70.0
       1019     66.0
       1020     11.0
       1021      6.0
       1022      6.0
       1023      3.0
       1024      8.0
       1025      3.0
       1026     13.0
       1027      2.0
       1029      1.0
       1030      2.0
       1031      2.0
...              ...
8001   6520      1.0
       6524      1.0
       6526      1.0
       6528      1.0
       6530      6.0
       6531      2.0
       6538      1.0
       6543      1.0
       6564      2.0
       6566      1.0
       6579      1.0
       6581      3.0
       6585      1.0
       6586      1.0
       6588      1.0
       6599      1.0
       6609      1.0
       6615     15.0
       6616     17.0
       6617     55.0
       6618     95.0
       6619    262.0
       6620    520.0
       6621    160.0
       6622      4.0
       6623      2.0
       6624      3.0
       6626     26.0
       8000      1.0
       8001     35.0

[381942 rows x 1 columns]

In [18]:
ex_skimdistf = os.path.join('example_data', 'ex_skimdist_1.csv')
ex_skimdist = Matrix(pd.DataFrame.from_csv(ex_skimdistf, index_col=[0,1]))
ex_skimdist

meters
zone_O zone_D             
1001   1002      395.00003
       1003      625.00000
       1004     1045.00012
       1005     1225.00012
       1006     2404.99951
       1007      784.99988
       1008     1345.00000
       1009     1699.99976
       1010     1610.00012
       1011     1965.00000
       1012     2330.00000
       1013     2930.00024
       1014     2204.99976
       1015     3294.99902
       1016     2945.00000
       1017     3150.00000
       1018     3139.99951
       1019     4160.00000
       1020     3845.00000
       1021     3789.99927
       1022     4579.99951
       1023     4855.00098
       1024     4950.00000
       1025     5294.99951
       1026     5150.00000
       1027     6894.99902
       1028     6795.00000
       1029     7184.99951
       1030     9655.00098
       1031     8305.00000
...                    ...
8001   6600    53748.96094
       6601    54353.96094
       6602    54193.96875
       6604    68178.39063
       6605    80427.67188
       6608    56685.00000
       6609    55510.00000
       6610    59349.99219
       6611    61135.00000
       6612    57930.13672
       6613    50023.85547
       6614    50455.14063
       6615     9334.99805
       6616     6770.00000
       6617     7734.44287
       6618     8629.55566
       6619     4629.99951
       6620     2805.00000
       6621     3165.00000
       6622    11105.00000
       6623    37805.55469
       6624    37575.75781
       6625    47523.96875
       6626     5155.00000
       6627    39772.79297
       6628    27860.14648
       6629    60453.96484
       6630    68531.44531
       6631    25420.94141
       8000    61044.99609

[1496950 rows x 1 columns]

In [19]:
ex_TLD = TLD_SingleDist(ex_matrix, ex_skimdist, 5)
ex_TLD

,T
meters,
0,0.0
5,322828.0
205,84.0
210,16.0
215,3.0
235,24.0
245,62.0
255,21.0
260,69.0


In [20]:
ex_TLD.sum()

T    4864971.0
dtype: float64

In [21]:
from MatrixExamples import mat

In [22]:
mat

T1  T2  T3
O D            
1 1   0   1   0
  2   1   2   6
  3   1   3  10
  4   0   4  12
  5   1   5  12
  6   1   6  10
  7   0   7   6
2 1   1   8   0
  2   1   9   0
  3   0  10  10
  4   1  11  12
  5   1  12  12
  6   0  13  10
  7   1  14   6
3 1   1  15   0
  2   0  16   6
  3   1  17   0
  4   1  18  12
  5   0  19  12
  6   1  20  10
  7   1  21   6
4 1   0  22   0
  2   1  23   6
  3   1  24  10
  4   0  25   0
  5   1  26  12
  6   1  27  10
  7   0  28   6
5 1   1  29   0
  2   1  30   6
  3   0  31  10
  4   1  32  12
  5   1  33   0
  6   0  34  10
  7   1  35   6
6 1   1  36   0
  2   0  37   6
  3   1  38  10
  4   1  39  12
  5   0  40  12
  6   1  41   0
  7   1  42   6
7 1   0  43   0
  2   1  44   6
  3   1  45  10
  4   0  46  12
  5   1  47  12
  6   1  48  10
  7   0  49   0

In [23]:
dst = mat.copy()
dst['T1'] = (dst.index.get_level_values(0)**2 - dst.index.get_level_values(1)**2)**2
dst['T2'] = dst['T1'] / dst.index.get_level_values(0)
dst['T2'] = dst['T1'] / dst.index.get_level_values(1)
dst.columns = 'D1 D2 D3'.split()
dst

D1           D2  D3
O D                       
1 1     0     0.000000   0
  2     9     4.500000   6
  3    64    21.333333  10
  4   225    56.250000  12
  5   576   115.200000  12
  6  1225   204.166667  10
  7  2304   329.142857   6
2 1     9     9.000000   0
  2     0     0.000000   0
  3    25     8.333333  10
  4   144    36.000000  12
  5   441    88.200000  12
  6  1024   170.666667  10
  7  2025   289.285714   6
3 1    64    64.000000   0
  2    25    12.500000   6
  3     0     0.000000   0
  4    49    12.250000  12
  5   256    51.200000  12
  6   729   121.500000  10
  7  1600   228.571429   6
4 1   225   225.000000   0
  2   144    72.000000   6
  3    49    16.333333  10
  4     0     0.000000   0
  5    81    16.200000  12
  6   400    66.666667  10
  7  1089   155.571429   6
5 1   576   576.000000   0
  2   441   220.500000   6
  3   256    85.333333  10
  4    81    20.250000  12
  5     0     0.000000   0
  6   121    20.166667  10
  7   576    82.285714   6
6 1  1225  1225.000000   0
  2  1024   512.000000   6
  3   729   243.000000  10
  4   400   100.000000  12
  5   121    24.200000  12
  6     0     0.000000   0
  7   169    24.142857   6
7 1  2304  2304.000000   0
  2  2025  1012.500000   6
  3  1600   533.333333  10
  4  1089   272.250000  12
  5   576   115.200000  12
  6   169    28.166667  10
  7     0     0.000000   0

In [24]:
TLD_single = TLD_SingleDist(mat,dst,5)
TLD_single

,T1,T2,T3
D1,,,
0,0.0,0.0,0.0
5,4.0,175.0,0.0
10,2.0,10.0,6.0
30,0.0,26.0,16.0
50,2.0,42.0,22.0
65,2.0,18.0,10.0
85,2.0,58.0,24.0
125,0.0,74.0,22.0
145,2.0,34.0,18.0


In [25]:
TLD_multi = TLD_MultiDist(mat,dst,5)
TLD_multi

,T1,T2,T3
0,0.0,0.0,0.0
5,4.0,177.0,0.0
10,2.0,18.0,72.0
15,NaN,34.0,264.0
20,NaN,50.0,NaN
25,NaN,151.0,NaN
30,0.0,48.0,NaN
40,NaN,11.0,NaN
50,2.0,NaN,NaN
55,NaN,19.0,NaN


In [26]:
mat.sum()

T1      32
T2    1225
T3     336
dtype: int64

In [27]:
TLD_single.sum()

T1      32.0
T2    1225.0
T3     336.0
dtype: float64

In [28]:
TLD_multi.sum()

T1      32.0
T2    1225.0
T3     336.0
dtype: float64

In [29]:
normalize_TLD(TLD_multi).sum()

T1    1.0
T2    1.0
T3    1.0
dtype: float64

In [30]:
band_agg_TLD(TLD_multi, 10).sum()

T1      32.0
T2    1225.0
T3     336.0
dtype: float64

In [152]:
OutputName = os.path.join('example_outputs', 'TLD.png')
TLD_to_JPG(TLD_multi, OutputName=OutputName)

In [151]:
oFileNamePattern = os.path.join('example_outputs', 'TLD_{}.png')
TLD_cols_to_JPGs(TLD_multi, oFileNamePattern=oFileNamePattern)